In [2]:
from IPython.display import display, HTML

display(HTML("""
<div>
  <label for="experimentId">Experiment ID:</label>
  <input type="text" id="experimentId" value="exp123" />
  
  <label for="duration">Duration:</label>
  <input type="number" id="duration" value="10" />

  <button id="startBtn">Start Simulation</button>
  <pre id="output" style="background: #f0f0f0; padding: 10px;"></pre>
</div>

<script type="text/javascript">
(async function() {
  const output = document.getElementById("output");

  function log(msg) {
    output.textContent += msg + "\\n";
  }

  function compressAndEncode(obj) {
    const jsonString = JSON.stringify(obj);
    const encoder = new TextEncoder();
    const uint8Data = encoder.encode(jsonString);
    const compressed = pako.gzip(uint8Data); // use pako for gzip
    const base64 = btoa(String.fromCharCode.apply(null, compressed));
    return base64;
  }

  function decodeAndDecompress(base64str) {
    const binaryString = atob(base64str);
    const charData = binaryString.split('').map(x => x.charCodeAt(0));
    const binData = new Uint8Array(charData);
    const decompressed = pako.ungzip(binData, { to: 'string' });
    return JSON.parse(decompressed);
  }

  document.getElementById("startBtn").onclick = function() {
    const experimentId = document.getElementById("experimentId").value;
    const duration = document.getElementById("duration").value;
    const uri = `ws://localhost:8080/api/v1/core/run-simulation?experiment_id=${experimentId}&duration=${duration}&overwrite=false`;

    const ws = new WebSocket(uri, []);
    const apiKey = "test";

    ws.onopen = () => {
      log("WebSocket connected.");

      const message = {
        auth: apiKey,
        message: "start"
      };

      const encoded = compressAndEncode(message);
      ws.send(encoded);
      log("Sent: " + JSON.stringify(message));
    };

    ws.onmessage = (event) => {
      try {
        const decoded = decodeAndDecompress(event.data);
        log("Received: " + JSON.stringify(decoded));
      } catch (e) {
        log("Error decoding message: " + e);
      }
    };

    ws.onerror = (err) => {
      log("WebSocket error: " + err.message);
    };

    ws.onclose = () => {
      log("WebSocket closed.");
    };
  };
})();
</script>

<!-- Load pako.js for gzip support -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/pako/2.1.0/pako.min.js"></script>
"""))
